# 👮 Agente Cidadão Inteligente - ACI

### 👉 Sistema de alerta e informação sobre recursos e serviços sociais

---

#### O **Agente Cidadão Inteligente (ACI)** é um ChatBot desenvolvido com o propósito de democratizar o acesso à informação sobre recursos e serviços sociais cruciais para os cidadãos de diversas regiões do Brasil. Buscamos fornecer detalhes sobre a **disponibilidade**, **localização** e **formas de acesso** a serviços nas áreas de saúde, assistência social, educação e muito mais. Através de uma interface conversacional intuitiva, o ACI se torna uma ferramenta poderosa para conectar as pessoas aos recursos de que necessitam, promovendo assim um maior bem-estar e inclusão social em nossa comunidade.  
Para mais detalhes, siga as instruções do README, presente no repositório do [**GitHub**](https://github.com/CesarImperas/acesso-cidadao-inteligente).

---
Desenvolvido por [**Caio Cesar Vieira**](https://www.linkedin.com/in/caiocesarvieira/)  
Com ajuda do [**Google Gemini**](https://gemini.google.com/app?hl=pt-PT)  
Em parceria com a "Imersão IA Alura + Google Gemini" ([**Alura**](https://www.alura.com.br/))

---
**Versão**: $1.0$

---

### Inicialização do Ambiente de Execução
* Importação de bibliotecas importantes como `os`, `google-genai`, `google-adk` e `IPython` (`display` e `Markdown`).
* Inicialização das funções utilizadas pelo ChatBot ACI

In [1]:
%pip install -q google-genai google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/

In [32]:
import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types

from IPython.display import display, Markdown

import requests
import warnings

warnings.filterwarnings("ignore")

# Configurar a API Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [35]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(text)

# Função para exibir mensagens formatadas
def exibir_mensagem(role=None, texto=""):
    if role == "Você":
        display(Markdown(f"###**😃 Você:** {texto}"))
    elif role == "ACI":
        display(to_markdown(f"###**👮 ACI:** {texto}"))
    else:
        display(Markdown(texto))

# --- Agente ACI ---
def agente_ci(localizacao):
    agente = Agent(
        name="agente_cidadao_inteligente",
        model="gemini-2.5-flash-preview-04-17-thinking", # Modelo Gemini - em casos de falha, use o modelo "gemini-2.0-flash"
        description=f"Agente especializado em informações sobre serviços sociais em {localizacao}.",
        instruction=f"""
            Você é um agente de informações sobre serviços sociais em {localizacao}.
            Sua tarefa é fornecer aos usuários detalhes precisos e atualizados sobre os serviços que eles solicitam.
            Use a ferramenta de busca do Google (google_search) para encontrar informações como:
            - Endereços e contatos dos locais que oferecem o serviço.
            - Horários de funcionamento.
            - Documentos necessários.
            - Critérios de elegibilidade.
            - Outras informações relevantes.
            Responda de forma clara, concisa e amigável. Insira emojis que facilitem a compreensão do usuário, em determinada resposta.
            Se não encontrar a informação, informe ao usuário que você fará o possível para ajudar de outras formas.
            Insira links de referência para confiabilidade ou consultas por parte do usuário.
        """,
        tools=[google_search]
    )
    return agente

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    sources_response = ""

    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
        elif event.name == "tool_call":
            for tool_call in event.tool_calls:
                if tool_call.tool_name == "google_search":
                    search_result = tool_call.result
                    if search_result and search_result.results:
                        sources_response += "\n**Referências:**\n"
                        for result in search_result.results:
                            sources_response += f"[{result.title}]({result.url})\n"
    return final_response + sources_response

### Vamos conversar com o ACI❓❗

In [36]:
# Função principal para executar o chat
def executar_chat_aci():
    """
    Executa o chat do Agente Cidadão Inteligente.

    Função principal, com entrada e saída, formatadas com o padrão Markdown.
    """

    # Esperando uma localização (válida) do usuário para início do agente ACI
    exibir_mensagem(texto="Por favor, insira a sua localização: ")
    localizacao = input().strip()

    # Tratamento com localização vazia
    if not localizacao:
      exibir_mensagem("ACI", "Entrada vazia ou inválida. Tente novamente!")
      return


    exibir_mensagem(texto="## 👮🧠 Bem-vindo ao Agente Cidadão Inteligente (ACI)")
    exibir_mensagem(texto=f"###Estou aqui para te ajudar com informações sobre serviços sociais em {localizacao}.")

    exibir_mensagem(texto="###❓ Como posso te ajudar hoje?  ")

    # Instanciar o agente
    agente_ci_var = agente_ci(localizacao)
    contexto_conversa = []
    MAX_CONTEXTO = 10

    while True:
        exibir_mensagem(role="Você")
        mensagem_usuario = input().strip()

        # Tratamento com mensagem do usuário vazia
        if not mensagem_usuario:
          exibir_mensagem("ACI", "Entrada vazia ou inválida. Tente novamente!")
          continue

        # Sair da conversação
        if mensagem_usuario.lower() in ["sair", "encerrar", "fim", "tchau", "xau", "valeu", "vlw"]:
            exibir_mensagem("ACI", "Ok, encerrando o chat. Espero ter ajudado!")
            break

        resposta_agente = call_agent(agente_ci_var, mensagem_usuario)
        exibir_mensagem("ACI", resposta_agente)

        # Adicionar a interação ao contexto
        contexto_conversa.append({"role": "user", "content": mensagem_usuario})
        contexto_conversa.append({"role": "ACI", "content": resposta_agente})

        # Limitar o tamanho do contexto - Histórico
        if len(contexto_conversa) > MAX_CONTEXTO:
            contexto_conversa.pop(0)
            contexto_conversa.pop(0)

# Executar o chat
if __name__ == "__main__":
    exibir_mensagem(texto="🚀 Iniciando o chat com o Agente Cidadão Inteligente! 🚀")
    executar_chat_aci()

🚀 Iniciando o chat com o Agente Cidadão Inteligente! 🚀

Por favor, insira a sua localização: 

campina grande


## 👮🧠 Bem-vindo ao Agente Cidadão Inteligente (ACI)

###Estou aqui para te ajudar com informações sobre serviços sociais em campina grande.

...

tem uma casa da cidadania no shopping partage?


###**👮 ACI:** Sim, Campina Grande possui uma unidade da Casa da Cidadania que funciona no Partage Shopping. 🎉 Essa unidade começou a operar no Partage Shopping enquanto a sede localizada no Açude Velho passa por reformas.

**O que você precisa saber:**

*   **Localização:** A Casa da Cidadania está temporariamente funcionando no Partage Shopping.
*   **Serviços:** Alguns dos serviços disponíveis nesta unidade incluem atendimentos do Procon-PB, Polícia Federal, Sine-PB, 2ª via do CPF e Carteira de Trabalho Digital. Serviços como emissão de carteira de identidade (RG), Cagepa, Receita Estadual, Defensoria Pública, INSS e Secretaria de Estado da Administração que antes eram no Açude Velho, também estão disponíveis no Partage Shopping durante o período de reforma.
*   **Agendamento:** Para emissão de RG, o agendamento é feito exclusivamente pelo Portal da Cidadania, utilizando a SenhaGov. Para outros serviços, o agendamento pode ser presencial.
*   **Horário de Funcionamento:** O funcionamento da unidade no Partage Shopping é das 8h às 17h. (Vale notar que em agosto de 2023 o horário informado era das 8h às 17h, acompanhando o horário comercial, e não o do shopping.) É sempre bom confirmar o horário atual.
*   **Próxima Mudança:** A Casa da Cidadania de Campina Grande passará por uma nova mudança em breve, para um espaço no Shopping SitMix, localizado na rua Getúlio Vargas, no Centro. A previsão de mudança era até o final de abril ou início de maio.

É importante ficar atento às notícias locais para confirmar se a mudança para o Shopping SitMix já ocorreu. 😊


###**😃 Você:** 

vlw


###**👮 ACI:** Ok, encerrando o chat. Espero ter ajudado!